## Import des bibliothèques 

In [10]:
import os
import sys
sys.path.append("../src")

from Corpus import Corpus
import praw
from datetime import datetime
import urllib.request
import xmltodict
import ssl
import certifi

## Extraction de données depuis Reddit

In [ ]:

def extraire_reddit(corpus, theme, mot_cles=None):
    if mot_cles is None:
        mot_cles = [
            'climate', 'climatechange', 'environment', 'sustainability',
            'carbonfootprint', 'ecology', 'greenenergy', 'climateaction'
        ]
    
    print("\n--- Extraction Reddit ---")
    
    # Configuration Reddit API
    reddit = praw.Reddit(
        client_id='_qZmFflED2oGl3LmqZvqBg',
        client_secret='1wQi5AamyCOBlq5HQqbpu85PDRjySQ',
        user_agent='projet_api'
    )
    
    for mot in mot_cles:
        try:
            posts = reddit.subreddit(mot).top(limit=20)
            for post in posts:
                text = post.selftext.replace("\n", " ").strip()
                if text == "":
                    continue

                corpus.add_document(
                    titre=post.title,
                    auteur=str(post.author) if post.author else "Unknown",
                    date=datetime.fromtimestamp(post.created_utc).strftime("%Y-%m-%d"),
                    url=f"https://reddit.com{post.permalink}",
                    texte=text
                )
                print(f"  ✓ Reddit: {post.title[:50]}...")
                
        except Exception as e:
            print(f"  ✗ Erreur avec le subreddit '{mot}': {e}")
    
    return corpus


## Extraction de données scientifiques depuis ArXiv

In [8]:

def extraire_arxiv(corpus, theme, max_results=10):
    """
    Extrait les articles ArXiv et les ajoute au corpus.
    
    Args:
        corpus: Instance de Corpus
        theme: Thème de recherche
        max_results: Nombre max d'articles à récupérer
    """
    print("\n--- Extraction ArXiv ---")
    
    # Configuration SSL pour ArXiv
    ssl_context = ssl.create_default_context(cafile=certifi.where())
    url = f"https://export.arxiv.org/api/query?search_query=all:{theme}&start=0&max_results={max_results}"
    
    try:
        # Récupération des données
        response = urllib.request.urlopen(url, context=ssl_context)
        data = xmltodict.parse(response.read())
        
        entries = data["feed"]["entry"]
        if isinstance(entries, dict):
            entries = [entries]
        
        for entry in entries:
            # Gestion des auteurs
            auteurs = entry["author"]
            if isinstance(auteurs, dict):
                auteurs = [auteurs]
            auteur_str = ", ".join(a["name"] for a in auteurs)
            
            corpus.add_document(
                titre=entry["title"].strip(),
                auteur=auteur_str,
                date=entry["published"],
                url=entry["id"],
                texte=entry["summary"].strip()
            )
            print(f"  ✓ ArXiv: {entry['title'][:50]}...")
            
    except Exception as e:
        print(f"  ✗ Erreur avec ArXiv: {e}")
    
    return corpus

## Construction ou chargement du corpus principal

In [7]:
theme = "climatechange"
filename = f"{theme}_corpus_new.csv"

corpus = Corpus(f"Corpus_{theme}")

# --- Si le fichier existe : charger ---
if os.path.exists(filename):
    print(f"\nChargement du corpus depuis {filename}...")
    corpus.load(filename)
    print(f"Corpus chargé : {corpus.ndoc} documents, {corpus.naut} auteurs")

# --- Sinon : construire le corpus ---
else:
    print(f"\nAucun fichier trouvé. Construction du corpus '{theme}'...")
    
    # Appel des fonctions d'extraction
    corpus = extraire_reddit(corpus, theme)
    corpus = extraire_arxiv(corpus, theme, max_results=15)
    
    # Sauvegarde du corpus
    corpus.save(filename)
    print(f"\n✅ Corpus construit et sauvegardé : {corpus.ndoc} documents, {corpus.naut} auteurs")



Chargement du corpus depuis climatechange_corpus_new.csv...
Corpus chargé depuis climatechange_corpus_new.csv (28 documents)
Corpus chargé : 28 documents, 28 auteurs


In [12]:
if not os.path.exists(filename):
    print("\nSauvegarde du corpus...")
    corpus.save(filename)


# Vérification du corpus avec des documents spécialisés

## 🧪Test avec un document Reddit

In [5]:
import sys
sys.path.append("../src")

from Corpus import Corpus
from RedditDocument import RedditDocument
from ArxivDocument import ArxivDocument

corpus = Corpus("Test")

# Ajout manuel d'un document Reddit
r = RedditDocument(
    titre="Mon post Reddit",
    auteur="Alice",
    date="2024-11-10",
    url="https://reddit.com/r/python",
    texte="Un post intéressant",
    nb_comments=120
)

corpus.id2doc["doc_0"] = r
corpus.ndoc += 1

corpus.afficher_sources()


doc_0 : Reddit - Mon post Reddit


## 🧪 Test avec un document ArXiv personnalisé

In [4]:
a = ArxivDocument(
    titre="Article d'IA",
    auteur="Yann LeCun",
    date="2023-09-02",
    url="https://arxiv.org/abs/1234.5678",
    texte="Résumé...",
    co_auteurs=["Hinton", "Bengio"]
)

corpus.id2doc["doc_1"] = a
corpus.ndoc += 1
corpus.afficher_sources()


doc_0 : Reddit - Mon post Reddit
doc_1 : Arxiv - Article d'IA


# Tests fonctionnels – TD6

## 📝 Création d’un corpus de test

In [28]:
corpus = Corpus("Test TD6")

corpus.add_document(
    "Climat et pollution",
    "Alice",
    "2024-01-10",
    "http://test1.com",
    "Le changement climatique est rapide. Pollution et climat."
)

corpus.add_document(
    "Energie renouvelable",
    "Bob",
    "2024-01-11",
    "http://test2.com",
    "L'énergie renouvelable lutte contre le changement climatique."
)


## 🔍 Recherche textuelle et concordancier

In [29]:
corpus.search("climatique")

[('doc_0', <document.Document at 0x1acf48bcd10>),
 ('doc_1', <document.Document at 0x1acf48bc310>)]

In [30]:
for c in corpus.concorde("climatique"):
    print(c)

('Le changement', 'climatique', 'est rapide. Pollution et clim', 'doc_0')
('le lutte contre le changement', 'climatique', '.', 'doc_1')


## 🧹 Nettoyage et normalisation des textes

In [31]:
print("BEFORE:", corpus.id2doc["doc_0"].texte)
corpus.nettoyer_texte()
print("AFTER:", corpus.id2doc["doc_0"].texte)


BEFORE: Le changement climatique est rapide. Pollution et climat.
AFTER: le changement climatique est rapide pollution et climat


## 📚 Construction du vocabulaire

In [32]:
corpus.build_vocab()


['changement',
 'climat',
 'climatique',
 'contre',
 'est',
 'et',
 'l',
 'le',
 'lutte',
 'pollution',
 'rapide',
 'renouvelable',
 'énergie']

## 📊 Analyse statistique du corpus

### Fréquence des termes (TF)

In [33]:
corpus.term_frequency()


,term_frequency
le,2
changement,2
climatique,2
est,1
rapide,1
pollution,1
et,1
climat,1
l,1
énergie,1


### Fréquence documentaire (DF)

In [34]:
corpus.document_frequency()

,document_frequency
changement,2
climat,1
climatique,2
contre,1
est,1
et,1
l,1
le,2
lutte,1
pollution,1


# TD7 – Vectorisation et moteur de recherche

## Construction de la matrice Documents × Mots

In [ ]:
import numpy as np
#transformer les documents en une matrice Documents × Mots
vocab = corpus.build_vocab()
print("\nVocabulaire:", vocab)


Vocabulaire: ['_update', 'a', 'able', 'about', 'absolute', 'absolutely', 'ac', 'accelerating', 'accepted', 'according', 'accurate', 'across', 'act', 'action', 'activists', 'actual', 'actually', 'added', 'adding', 'addition', 'additional', 'address', 'adjust', 'adk', 'advances', 'advantage', 'aenocyon', 'affected', 'affecting', 'after', 'aftermath', 'ago', 'agriculture', 'air', 'al', 'albedo', 'albeit', 'all', 'allegany', 'allowed', 'almond', 'almost', 'along', 'already', 'also', 'always', 'am', 'amount', 'an', 'analog', 'analysis', 'ancient', 'and', 'animal', 'animals', 'annual', 'another', 'answer', 'any', 'anymore', 'anyone', 'anything', 'appealing', 'apply', 'appreciate', 'approaches', 'are', 'area', 'areas', 'argument', 'arizona', 'around', 'arrays', 'article', 'articles', 'as', 'ashwani', 'asia', 'ask', 'assessment', 'assume', 'assumption', 'at', 'atmosphere', 'attention', 'attract', 'attracting', 'authorities', 'auto', 'automatically', 'avoiding', 'awareness', 'b', 'ba', 'back',

## Moteur de recherche TF-IDF

In [19]:
from SearchEngine import SearchEngine

engine = SearchEngine(corpus, mode="tfidf")
engine.search("climate warming pollution water", top=5)


,id,titre,score
27,doc_27,Climate Interactive & MIT Launch [Free] Traini...,0.196290
55,doc_55,Climate Interactive & MIT Launch [Free] Traini...,0.196290
31,doc_31,Joe Rogan Fact-Checks Bernie Sanders on Climat...,0.179234
3,doc_3,Joe Rogan Fact-Checks Bernie Sanders on Climat...,0.179234
0,doc_0,"Holy sh*t - for the first time EVER, monsoon w...",0.086982
